In [2]:
using BenchmarkTools
using Distributed
using Plots
using Profile
using ProfileVega
using PyCall
using ScikitLearn
using Traceur
using TimerOutputs

In [3]:
@sk_import datasets: (fetch_covtype)

PyObject <function fetch_covtype at 0x7fce29bb93a0>

In [4]:
digits_data = fetch_covtype();

In [5]:
X_digits = abs.(digits_data["data"]);
X_digits = transpose(X_digits);

In [6]:
d, n = size(X_digits);

In [7]:
struct RandomGreedy
    X::Matrix{Float64}
end

struct NaiveGreedy
    X::Matrix{Float64}
end

struct Result
    ranking::Vector{Int32}
    gains::Vector{Float64}
end

In [8]:
function lexsort(a, b, rev=false) 
    idxs = sortperm(a, alg=MergeSort, rev=rev)
    return idxs[sortperm(b[idxs], alg=MergeSort, rev=rev)]
end;

In [15]:
@btime begin
    idxs = 1:n |> collect
#     global n = size(idxs)[1]
#     gains = zeros(n)
    current_values = zeros(Float64, d)
    current_concave_values = sqrt.(current_values)
    current_concave_values_sum = sum(current_concave_values)

#     gains = sum(pmap(x -> sqrt.(current_values+x), eachrow(X_digits)));
#     Threads.@threads for i in 1:n
#         idx = idxs[i]
#         @fastmath gains[i] = sum(sqrt.(current_values + view(X_digits, idx, :)))
#     end
#     X_current = X_digits .+ current_values
    gains = zeros(Float64, n);
    calculate_gains!(X_digits, gains, current_values, idxs, current_concave_values_sum);
#     gains = calculate_gains!(X_digits, gains, current_values, idxs, current_concave_values_sum);
#     i = 1
#     @distributed for row in eachrow(X_digits)
#         gains[i] = sum(sqrt.(current_values + row))
#         i += 1
#     end
end

  14.603 ms (92 allocations: 8.87 MiB)


581012-element Vector{Float64}:
 221.97327539529405
 219.8966797240838
 271.1471644897934
 274.1822937228027
 215.14557840730572
 216.15354296606503
 226.97207155280068
 224.90252282136385
 231.9978510673305
 228.16724685228837
 238.8754089929368
 274.2626594102293
 266.2365723322574
   ⋮
 169.65537402249007
 168.53603624501983
 166.7954782546634
 164.993267619306
 164.0323204798027
 163.57708363018048
 162.99973719540802
 162.20583123960742
 159.84616192019166
 158.9693746251935
 159.37684265706847
 157.4720088161721

In [14]:
const to = TimerOutput();

For loop
881.714 ms (8131620 allocations: 1023.93 MiB)

For loop using eachrows
615.203 ms (5809105 allocations: 731.39 MiB)

Parallel for loop
108.468 ms (6970150 allocations: 997.35 MiB)
--with view:
91.984 ms (6970148 allocations: 722.51 MiB)
--with fastmath:
78.470 ms (6970148 allocations: 722.51 MiB)
--as function
70.131 ms (1743086 allocations: 913.15 MiB)
--using column first orientation
52.535 ms (2323594 allocations: 629.45 MiB)
--precalculating array
89.790 ms (1742586 allocations: 567.39 MiB)
--kernel-function and eachindex (eachindex within the main function was worse ~53ms)
51.915 ms (1162079 allocations: 616.16 MiB)

Map over each row
357.097 ms (1743047 allocations: 913.15 MiB)

In [10]:
function get_gains!(X, current_values, idxs, gains)
    @inbounds Threads.@threads for i in eachindex(idxs)
        s = 0.0
        for j in eachindex(current_values)
            s += @fastmath sqrt(current_values[j] + X[j, idxs[i]])
        end
        gains[i] = s
    end
end;

In [11]:
function calculate_gains!(X, gains, current_values, idxs, current_concave_value_sum)
    get_gains!(X, current_values, idxs, gains)
    
    gains .-= current_concave_value_sum
    return gains
end;

In [12]:
function fit(optimizer::RandomGreedy, k)
    d, n = size(optimizer.X)
    sample_cost = ones(Float64, n)

    cost = 0.0

    ranking = Int32[]
    total_gains = Float64[]

    mask = zeros(Int8, n)
    current_values = zeros(Float64, d)
    current_concave_values = sqrt.(current_values)
    current_concave_values_sum = sum(current_concave_values)

    idxs = 1:n

    while cost < k
        idx = rand(idxs)

        if cost + sample_cost[idx] > k
            continue
        end

        cost += sample_cost[idx]

        # Calculate gains
        gains = zeros(Float64, size(idxs)[1])
        gains = calculate_gains!(optimizer.X, gains, current_values, Vector([idx]), current_concave_values_sum)
        gain = gains[1]

        # Select next
        current_values += view(optimizer.X, :, idx)
        current_concave_values = sqrt.(current_values)
        current_concave_values_sum = sum(current_concave_values)

        push!(ranking, idx)
        push!(total_gains, gain)

        mask[idx] = 1
        idxs = findall(==(0), mask)
    end
    return Result(ranking, total_gains)
end;

In [23]:
function fit(optimizer::NaiveGreedy, k, sample_cost)
    @timeit to "intro" begin
        d, n = size(optimizer.X)
        
        cost = 0.0

        ranking = Int32[]
        total_gains = Float64[]

        mask = zeros(Int8, n)
        current_values = zeros(Float64, d)
        current_concave_values = sqrt.(current_values)
        current_concave_values_sum = sum(current_concave_values)

        idxs = 1:n
    end

    gains = zeros(Float64, size(idxs)[1])
    @timeit to "while loop" begin
        while cost < k
            gains = @timeit to "calc_gains" calculate_gains!(optimizer.X, gains, current_values, idxs, current_concave_values_sum)
            
            if sample_cost != nothing
                gains ./= sample_cost[idxs]
                idx_idxs = @timeit to "lexsort" lexsort(gains, 1:size(gains)[1], true)

                @timeit to "select_idx" begin
                    for i in 1:size(idx_idxs)[1]
                        global idx = idx_idxs[i]
                        global best_idx = idxs[idx]
                        if cost + sample_cost[best_idx] <= k
                            break
                        end
                    end
                end
                curr_cost = sample_cost[best_idx]
            else
                global idx = argmax(gains)
                global best_idx = idxs[idx]
                curr_cost = 1.
            end
            
            if cost + curr_cost > k
                break
            end
                
            @timeit to "select_idx" begin
                cost += curr_cost
                # Calculate gains
                gain = gains[idx] * curr_cost
            end
            
            @timeit to "select_next" begin
                # Select next
                current_values += view(optimizer.X, :, best_idx)
                current_concave_values .= sqrt.(current_values)
                current_concave_values_sum = sum(current_concave_values)

                push!(ranking, best_idx)
                push!(total_gains, gain)

                mask[best_idx] = 1
                idxs = findall(==(0), mask)
            end
        end
    end
    return Result(ranking, total_gains)
end;

In [12]:
const to = TimerOutput();

In [20]:
k = 1000;

In [24]:
opt1 = NaiveGreedy(X_digits);
res1 = fit(opt1, k, nothing);
# bench1 = @benchmark fit(opt1, k);
# t1 = bench1.times[1]/1e9

In [25]:
show(to)

 ────────────────────────────────────────────────────────────────────────
                                 Time                   Allocations      
                         ──────────────────────   ───────────────────────
    Tot / % measured:          254s / 16.3%           24.2GiB / 76.5%    

 Section         ncalls     time   %tot     avg     alloc   %tot      avg
 ────────────────────────────────────────────────────────────────────────
 while loop           3    41.2s   100%   13.7s   18.5GiB  100%   6.17GiB
   calc_gains     2.10k    21.1s  51.3%  10.1ms   18.2MiB  0.10%  8.89KiB
   select_next    2.10k    16.6s  40.4%  7.93ms   18.5GiB  100%   9.01MiB
   select_idx     2.10k   7.91ms  0.02%  3.77μs   67.8KiB  0.00%    33.1B
 intro                3   96.4μs  0.00%  32.1μs   1.67MiB  0.01%   569KiB
 ────────────────────────────────────────────────────────────────────────

In [18]:
show(to)

 ────────────────────────────────────────────────────────────────────────
                                 Time                   Allocations      
                         ──────────────────────   ───────────────────────
    Tot / % measured:          109s / 18.2%           60.9GiB / 98.9%    

 Section         ncalls     time   %tot     avg     alloc   %tot      avg
 ────────────────────────────────────────────────────────────────────────
 while loop           2    19.8s   100%   9.91s   60.2GiB  100%   30.1GiB
   calc_gains       101    15.9s  80.2%   157ms   58.9GiB  97.8%   597MiB
   select_next      100    2.71s  13.7%  27.1ms    902MiB  1.46%  9.02MiB
   select_idx       100   5.32ms  0.03%  53.2μs   5.33KiB  0.00%    54.6B
 intro                2    147μs  0.00%  73.4μs   1.11MiB  0.00%   569KiB
 ────────────────────────────────────────────────────────────────────────

In [ ]:
opt0 = RandomGreedy(X_digits);
res0 = fit(opt0, k);
bench0 = @benchmark fit(opt0, k);
t0 = bench0.times[1]/1e9

In [ ]:
p1 = plot(cumsum(res0.gains), label="Random")
plot!(cumsum(res1.gains), label="Naive")
ylabel!("F(S)", fontsize=12)
xlabel!("Subset Size", fontsize=12)

labels = ["Random", "Naive"]
p2 = bar(labels, [t0, t1], labels=labels, legend=false)
ylabel!("Time (s)")

plot(p1, p2, layout = (1, 2))
plot!(size=(900,500))